# Direct forecasting

[Forecasting with Machine Learning - Course](https://www.trainindata.com/p/forecasting-with-machine-learning)

Load the retail sales data set located in Facebook's Prophet Github repository and use **direct forecasting** to predict future sales. 

- We want to forecast sales over the next 3 months. 
- Sales are recorded monthly. 
- We assume that we have all data to the month before the first point in the forecasting horizon.

We will forecast using Scikit-learn in this exercise.

Follow the guidelines below to accomplish this assignment.

## Import required classes and functions

In [7]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.linear_model import Lasso
from sklearn.metrics import root_mean_squared_error
from sklearn.multioutput import MultiOutputRegressor
from sklearn.preprocessing import MinMaxScaler

from feature_engine.datetime import DatetimeFeatures
from feature_engine.imputation import DropMissingData
from feature_engine.timeseries.forecasting import (
    LagFeatures,
    WindowFeatures,
)
from feature_engine.pipeline import Pipeline

## Load data

In [8]:
url = "https://raw.githubusercontent.com/facebook/prophet/master/examples/example_retail_sales.csv"
df = pd.read_csv(url)
df.to_csv("example_retail_sales.csv", index=False)

df = pd.read_csv(
    "example_retail_sales.csv",
    parse_dates=["ds"],
    index_col=["ds"],
    nrows=160,
)

df = df.asfreq("MS")

df.head()

,y
ds,
1992-01-01,146376
1992-02-01,147079
1992-03-01,159336
1992-04-01,163669
1992-05-01,170068


## Create the target variable

In direct forecasting, we train a model per step. Hence, we need to create 1 target per step.

In [9]:
# The forecasting horizon.
horizon = 3

# Create an empty dataframe for the targets.
y = pd.DataFrame(index=df.index)

# Add each one of the steps in the horizon.
for h in range(horizon):
    y[f"h_{h}"] = df.shift(periods=-h, freq="MS")
    
y

,h_0,h_1,h_2
ds,,,
1992-01-01,146376,147079.0,159336.0
1992-02-01,147079,159336.0,163669.0
1992-03-01,159336,163669.0,170068.0
1992-04-01,163669,170068.0,168663.0
1992-05-01,170068,168663.0,169890.0
...,...,...,...
2004-12-01,387155,293261.0,295062.0
2005-01-01,293261,295062.0,339141.0
2005-02-01,295062,339141.0,335632.0


In [10]:
# Remove nan from target

y.dropna(inplace=True)

# align data to available target values

df = df.loc[y.index]

df.tail()

,y
ds,
2004-10-01,319726
2004-11-01,324259
2004-12-01,387155
2005-01-01,293261
2005-02-01,295062


## Split data into train and test

Leave data from 2004 onwards in the test set.

In [11]:
end_train = '2003-12-31'
X_train = df.loc[:end_train]
y_train = y.loc[:end_train]

# We need as many ddata points as the
# biggest lag.

# For a lag of 12:
begin_test = '2003-01-01'
X_test = df.loc[begin_test:]
y_test = y.loc[begin_test:]

In [13]:
X_train.tail()

,y
ds,
2003-08-01,317056
2003-09-01,293890
2003-10-01,304036
2003-11-01,301265
2003-12-01,357577


In [14]:
X_test.head()

,y
ds,
2003-01-01,268328
2003-02-01,259051
2003-03-01,293693
2003-04-01,294251
2003-05-01,312389


## Set up regression model

We will use Lasso in this assignment.

In [15]:
# Lasso

lasso = MultiOutputRegressor(Lasso(random_state=0))

## Set up a feature engineering pipeline

Set up transformers from feature-engine and / or scikit- learn in a pipeline and test it to make sure the input feature table is the one you need for the forecasts.

We will use feature-engine because we are great fans of the library.

If you prefer pandas, as long as the input feature table is the one you expect, that is also a suitable alternative.

In [16]:
# Lag features.

lagf = LagFeatures(
    variables=["y"], # the input variables
    periods=[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12], # 12 lags
    missing_values="ignore",
)

# test
lagf.fit_transform(X_train)

,y,y_lag_1,y_lag_2,y_lag_3,y_lag_4,y_lag_5,y_lag_6,y_lag_7,y_lag_8,y_lag_9,y_lag_10,y_lag_11,y_lag_12
ds,,,,,,,,,,,,,
1992-01-01,146376,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1992-02-01,147079,146376.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1992-03-01,159336,147079.0,146376.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1992-04-01,163669,159336.0,147079.0,146376.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1992-05-01,170068,163669.0,159336.0,147079.0,146376.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2003-08-01,317056,309923.0,300998.0,312389.0,294251.0,293693.0,259051.0,268328.0,337006.0,290705.0,287540.0,273189.0,307256.0
2003-09-01,293890,317056.0,309923.0,300998.0,312389.0,294251.0,293693.0,259051.0,268328.0,337006.0,290705.0,287540.0,273189.0
2003-10-01,304036,293890.0,317056.0,309923.0,300998.0,312389.0,294251.0,293693.0,259051.0,268328.0,337006.0,290705.0,287540.0


In [17]:
# window features

winf = WindowFeatures(
    variables=["y"],     # the input variable
    window=[3, 6, 12],   # window size
    periods=1,           # move 1 month forward
    functions=["mean", "std"],
    missing_values="ignore",
    drop_original=True,  # drop demand when I dont need it anymore
)

# test
winf.fit_transform(X_train)

,y_window_3_mean,y_window_3_std,y_window_6_mean,y_window_6_std,y_window_12_mean,y_window_12_std
ds,,,,,,
1992-01-01,NaN,NaN,NaN,NaN,NaN,NaN
1992-02-01,NaN,NaN,NaN,NaN,NaN,NaN
1992-03-01,NaN,NaN,NaN,NaN,NaN,NaN
1992-04-01,150930.333333,7288.002218,NaN,NaN,NaN,NaN
1992-05-01,156694.666667,8604.621220,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...
2003-08-01,307770.000000,5992.935591,295050.833333,19264.649495,294527.416667,21381.590610
2003-09-01,309325.666667,8045.647664,304718.333333,9829.645195,295344.083333,22087.457335
2003-10-01,306956.333333,11864.515259,304751.166667,9785.683245,297069.166667,20980.443660


In [18]:
datetime_f = DatetimeFeatures(
    variables = "index",
    features_to_extract = ["month", "day_of_week"],
    drop_original=True,
)

datetime_f.fit_transform(X_train)

,y,month,day_of_week
ds,,,
1992-01-01,146376,1,2
1992-02-01,147079,2,5
1992-03-01,159336,3,6
1992-04-01,163669,4,2
1992-05-01,170068,5,4
...,...,...,...
2003-08-01,317056,8,4
2003-09-01,293890,9,0
2003-10-01,304036,10,2


In [20]:
# test transformation pipeline

pipe = Pipeline(
    [
        ("dt", datetime_f),
        ("lagf", lagf),
        ("winf", winf),
        ("dropna", DropMissingData()),
        ("scaler", MinMaxScaler()),
    ]
).set_output(transform="pandas")

pipe.fit(X_train, y_train)

Pipeline(steps=[('dt',
                 DatetimeFeatures(features_to_extract=['month', 'day_of_week'],
                                  variables='index')),
                ('lagf',
                 LagFeatures(missing_values='ignore',
                             periods=[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12],
                             variables=['y'])),
                ('winf',
                 WindowFeatures(drop_original=True, functions=['mean', 'std'],
                                missing_values='ignore', variables=['y'],
                                window=[3, 6, 12])),
                ('dropna', DropMissingData()), ('scaler', MinMaxScaler())])

In [21]:
# the data that will be used for training
# or for prediction later on:

Xt = pipe.transform(X_train)

Xt.head()

,month,day_of_week,y_lag_1,y_lag_2,y_lag_3,y_lag_4,y_lag_5,y_lag_6,y_lag_7,y_lag_8,y_lag_9,y_lag_10,y_lag_11,y_lag_12,y_window_3_mean,y_window_3_std,y_window_6_mean,y_window_6_std,y_window_12_mean,y_window_12_std
ds,,,,,,,,,,,,,,,,,,,,
1993-01-01,0.000000,0.666667,0.314313,0.114809,0.126087,0.077734,0.108480,0.105944,0.099380,0.106897,0.072662,0.049481,0.000000,0.000000,0.176767,0.474567,0.054520,0.527507,0.000000,0.135111
1993-02-01,0.090909,0.000000,0.016767,0.314313,0.114809,0.126087,0.077734,0.108480,0.105944,0.099380,0.106897,0.072662,0.064535,0.003688,0.131781,0.647640,0.034298,0.626264,0.004338,0.064022
1993-03-01,0.181818,0.000000,0.000000,0.016767,0.314313,0.114809,0.126087,0.077734,0.108480,0.105944,0.099380,0.106897,0.087349,0.067985,0.084536,0.757885,0.009700,0.717775,0.006245,0.028742
1993-04-01,0.272727,0.500000,0.108881,0.000000,0.016767,0.314313,0.114809,0.126087,0.077734,0.108480,0.105944,0.099380,0.121041,0.090715,0.000000,0.240742,0.016763,0.710842,0.013282,0.000454
1993-05-01,0.363636,0.833333,0.141072,0.108881,0.000000,0.016767,0.314313,0.114809,0.126087,0.077734,0.108480,0.105944,0.113644,0.124283,0.051152,0.307696,0.020161,0.714429,0.021386,0.000000


In [22]:
Xt.tail()

,month,day_of_week,y_lag_1,y_lag_2,y_lag_3,y_lag_4,y_lag_5,y_lag_6,y_lag_7,y_lag_8,y_lag_9,y_lag_10,y_lag_11,y_lag_12,y_window_3_mean,y_window_3_std,y_window_6_mean,y_window_6_std,y_window_12_mean,y_window_12_std
ds,,,,,,,,,,,,,,,,,,,,
2003-08-01,0.636364,0.666667,0.855108,0.807360,0.868301,0.771265,0.768279,0.582948,0.632579,1.000000,0.752294,0.735361,0.663992,0.843939,0.989725,0.124525,0.917524,0.646998,0.963520,0.673597
2003-09-01,0.727273,0.000000,0.893269,0.855108,0.807360,0.868301,0.771265,0.768279,0.582948,0.632579,1.000000,0.752294,0.739553,0.665231,1.000000,0.172572,0.987891,0.287318,0.969732,0.741360
2003-10-01,0.818182,0.333333,0.769333,0.893269,0.855108,0.807360,0.868301,0.771265,0.768279,0.582948,0.632579,1.000000,0.756217,0.740513,0.984352,0.261957,0.988130,0.285642,0.982852,0.635087
2003-11-01,0.909091,0.833333,0.823613,0.769333,0.893269,0.855108,0.807360,0.868301,0.771265,0.768279,0.582948,0.632579,1.000000,0.757116,0.971391,0.256062,1.000000,0.232952,0.993307,0.621534
2003-12-01,1.000000,0.000000,0.808789,0.823613,0.769333,0.893269,0.855108,0.807360,0.868301,0.771265,0.768279,0.582948,0.638398,1.000000,0.936627,0.107001,0.986505,0.218792,1.000000,0.608672


In [23]:
# we can also obtain the aligned target if 
# we wanted with feature-engine's pipeline:

Xt, yt = pipe.transform_x_y(X_train, y_train)

In [25]:
Xt.head()

,month,day_of_week,y_lag_1,y_lag_2,y_lag_3,y_lag_4,y_lag_5,y_lag_6,y_lag_7,y_lag_8,y_lag_9,y_lag_10,y_lag_11,y_lag_12,y_window_3_mean,y_window_3_std,y_window_6_mean,y_window_6_std,y_window_12_mean,y_window_12_std
ds,,,,,,,,,,,,,,,,,,,,
1993-01-01,0.000000,0.666667,0.314313,0.114809,0.126087,0.077734,0.108480,0.105944,0.099380,0.106897,0.072662,0.049481,0.000000,0.000000,0.176767,0.474567,0.054520,0.527507,0.000000,0.135111
1993-02-01,0.090909,0.000000,0.016767,0.314313,0.114809,0.126087,0.077734,0.108480,0.105944,0.099380,0.106897,0.072662,0.064535,0.003688,0.131781,0.647640,0.034298,0.626264,0.004338,0.064022
1993-03-01,0.181818,0.000000,0.000000,0.016767,0.314313,0.114809,0.126087,0.077734,0.108480,0.105944,0.099380,0.106897,0.087349,0.067985,0.084536,0.757885,0.009700,0.717775,0.006245,0.028742
1993-04-01,0.272727,0.500000,0.108881,0.000000,0.016767,0.314313,0.114809,0.126087,0.077734,0.108480,0.105944,0.099380,0.121041,0.090715,0.000000,0.240742,0.016763,0.710842,0.013282,0.000454
1993-05-01,0.363636,0.833333,0.141072,0.108881,0.000000,0.016767,0.314313,0.114809,0.126087,0.077734,0.108480,0.105944,0.113644,0.124283,0.051152,0.307696,0.020161,0.714429,0.021386,0.000000


In [26]:
yt.head()

,h_0,h_1,h_2
ds,,,
1993-01-01,153221,150087.0,170439.0
1993-02-01,150087,170439.0,176456.0
1993-03-01,170439,176456.0,182231.0
1993-04-01,176456,182231.0,181535.0
1993-05-01,182231,181535.0,183682.0


In [27]:
Xt.shape, yt.shape

((132, 20), (132, 3))

In [28]:
X_train.shape, y_train.shape

((144, 1), (144, 3))

## Test pipeline over test set

Ensure that the returned input feature table is suitable to forecast from `2004-01-01` onwards.

## Train a recursive forecaster

Now that we know that the pipeline works, we can train the forecaster.

You can take the feature table and target returned up to here to train the Lasso. 

Or, as we will do, you can add the Lasso within the pipeline.

# Forecast 3 months of sales

We'll start by forecasting 3 months of sales, starting at every single point of the test set.

This is the equivalent of backtesting without refit. More info in section 6!!

## Plot predictions vs actuals

Pick the first row of predictions and plot them against the real sales.

## Determine the RMSE

Pick the first row of predictions and calculate the RMSE

## Forecast next 3 months of sales

Predict the first 3 months of sales right after the end of the test set.

That is, starting on `2005-02-02`.